In [5]:
#Doc2Word method
import pandas as pd
train=pd.read_csv("/Users/shichangtai/Desktop/Kaggle/Bag of Wprds Meet Bags of Popcorn/labeledTrainData.tsv",header=0,delimiter='\t',quoting=3)
test=pd.read_csv("/Users/shichangtai/Desktop/Kaggle/Bag of Wprds Meet Bags of Popcorn/testData.tsv",header=0,delimiter='\t',quoting=3)
unlabeled_train=pd.read_csv("/Users/shichangtai/Desktop/Kaggle/Bag of Wprds Meet Bags of Popcorn/unlabeledTrainData.tsv",header=0,delimiter='\t',quoting=3)
print("Load %d labeled train data, %d unlabeled train data and %d test data" % (train["review"].size, unlabeled_train["review"].size, len(test['review'])))

Load 25000 labeled train data, 50000 unlabeled train data and 25000 test data


In [8]:
#Don't remove stop words in Word2Vec
# After experiment, we should not use porter stemmer in Word2Vec approach!
from bs4 import BeautifulSoup
import re
import nltk
from nltk.corpus import stopwords
def review_to_list(raw_review,remove_stopwords=False):
    #1. remove html
    raw_text=BeautifulSoup(raw_review,'lxml').get_text()
    #2. remove non-letter (maybe we can hold the number)
    only_letter_text=re.sub("[^a-zA-Z]"," ",raw_text)
    #3. split and lower case
    pure_text=only_letter_text.lower().split()
    #4. (optional) remove stop words
    if remove_stopwords:
        #stemmer=nltk.PorterStemmer()
        stopword=set(stopwords.words("english"))
        pure_text=[m for m in pure_text if m not in stopword]
    #5. return a list of words
    return pure_text

In [12]:
a=[1,2,3,4,5,6]
import random
random.shuffle(a)
print (random.sample(a,len(a)))

[3, 5, 4, 6, 1, 2]


In [13]:
#Doc2Vec method
import numpy as np
import random
from gensim.models.doc2vec import LabeledSentence, Doc2Vec
def review_to_doc(uid,review,remove_stopwords=False):
    labeledSent=LabeledSentence(review_to_list(review,remove_stopwords),["%s" % uid])
    return labeledSent

all_documents=[]
print("Parsing documents from training data...")
for ids,review in enumerate(train["review"]):
    all_documents.append(review_to_doc("train"+str(ids),review.decode('utf-8')))
print("Parsing documents from unlabeled training data...")
for ids,review in enumerate(unlabeled_train["review"]):
    all_documents.append(review_to_doc("unlabeled_train"+str(ids),review.decode('utf-8')))
print("Parsing documents from test data...")
for ids,review in enumerate(test["review"]):
    all_documents.append(review_to_doc("test"+str(ids),review.decode('utf-8')))
    
import logging
logging.basicConfig(format='%(asctime)s:%(levelname)s:%(message)s',level=logging.INFO)

num_features=500
min_word_count=1
num_workers=4
context=10
downsampling=1e-3

from gensim.models import doc2vec
print "Training model..."
model=Doc2Vec(workers=num_workers,\
                        size=num_features,min_count=min_word_count,\
                        window=context,sample=downsampling)
# gensim documentation suggests training over data set for multiple times
# by either randomizing the order of the data set or adjusting learning rate
model.build_vocab(all_documents)
for epoch in range(10):
    print("Iteration: "+str(epoch)+"!!!!!!!!!!!!!!")
    model.train(random.sample(all_documents,len(all_documents)))
#print(model.most_similar(positive=['woman', 'king'], negative=['man'], topn=10))
model.save("Doc2VectforNLPTraining")  

/usr/local/lib/python2.7/site-packages/gensim/utils.py:1015: UserWarning: Pattern library is not installed, lemmatization won't be available.
  warnings.warn("Pattern library is not installed, lemmatization won't be available.")


Parsing documents from training data...
Parsing documents from unlabeled training data...
Parsing documents from test data...


2016-11-03 15:46:41,452:INFO:collecting all words and their counts
2016-11-03 15:46:41,454:INFO:PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags


Training model...


2016-11-03 15:46:42,116:INFO:PROGRESS: at example #10000, processed 2385436 words (3606324/s), 51525 word types, 10000 tags
2016-11-03 15:46:42,783:INFO:PROGRESS: at example #20000, processed 4747357 words (3543868/s), 67812 word types, 20000 tags
2016-11-03 15:46:43,452:INFO:PROGRESS: at example #30000, processed 7099901 words (3520813/s), 81669 word types, 30000 tags
2016-11-03 15:46:44,149:INFO:PROGRESS: at example #40000, processed 9467428 words (3403222/s), 93388 word types, 40000 tags
2016-11-03 15:46:44,839:INFO:PROGRESS: at example #50000, processed 11865291 words (3476666/s), 103473 word types, 50000 tags
2016-11-03 15:46:45,523:INFO:PROGRESS: at example #60000, processed 14247872 words (3488512/s), 112174 word types, 60000 tags
2016-11-03 15:46:46,611:INFO:PROGRESS: at example #70000, processed 16608448 words (2172020/s), 119831 word types, 70000 tags
2016-11-03 15:46:47,277:INFO:PROGRESS: at example #80000, processed 18946288 words (3513098/s), 127893 word types, 80000 tags


...

2016-11-03 16:26:49,984:INFO:PROGRESS: at 0.29% examples, 256179 words/s, in_qsize 8, out_qsize 0
2016-11-03 16:26:51,032:INFO:PROGRESS: at 0.50% examples, 219432 words/s, in_qsize 7, out_qsize 0
2016-11-03 16:26:52,033:INFO:PROGRESS: at 0.80% examples, 237541 words/s, in_qsize 7, out_qsize 0
2016-11-03 16:26:53,040:INFO:PROGRESS: at 1.14% examples, 251587 words/s, in_qsize 7, out_qsize 0
2016-11-03 16:26:54,045:INFO:PROGRESS: at 1.46% examples, 258939 words/s, in_qsize 7, out_qsize 0
2016-11-03 16:26:55,046:INFO:PROGRESS: at 1.80% examples, 265280 words/s, in_qsize 8, out_qsize 0
2016-11-03 16:26:56,060:INFO:PROGRESS: at 2.13% examples, 269025 words/s, in_qsize 8, out_qsize 0
2016-11-03 16:26:57,078:INFO:PROGRESS: at 2.47% examples, 272561 words/s, in_qsize 7, out_qsize 0
2016-11-03 16:26:58,122:INFO:PROGRESS: at 2.80% examples, 274730 words/s, in_qsize 7, out_qsize 0
2016-11-03 16:26:59,129:INFO:PROGRESS: at 3.13% examples, 276934 words/s, in_qsize 7, out_qsize 0
2016-11-03 16:27:00,

Iteration: 8!!!!!!!!!!!!!!


2016-11-03 16:32:19,219:INFO:PROGRESS: at 0.30% examples, 259188 words/s, in_qsize 6, out_qsize 1
2016-11-03 16:32:20,219:INFO:PROGRESS: at 0.58% examples, 254805 words/s, in_qsize 7, out_qsize 0
2016-11-03 16:32:21,227:INFO:PROGRESS: at 0.92% examples, 265337 words/s, in_qsize 7, out_qsize 0
2016-11-03 16:32:22,232:INFO:PROGRESS: at 1.22% examples, 267743 words/s, in_qsize 8, out_qsize 0
2016-11-03 16:32:23,240:INFO:PROGRESS: at 1.55% examples, 271712 words/s, in_qsize 7, out_qsize 0
2016-11-03 16:32:24,262:INFO:PROGRESS: at 1.89% examples, 276093 words/s, in_qsize 8, out_qsize 0
2016-11-03 16:32:25,275:INFO:PROGRESS: at 2.22% examples, 278626 words/s, in_qsize 8, out_qsize 0
2016-11-03 16:32:26,329:INFO:PROGRESS: at 2.53% examples, 276341 words/s, in_qsize 8, out_qsize 0
2016-11-03 16:32:27,339:INFO:PROGRESS: at 2.81% examples, 272598 words/s, in_qsize 7, out_qsize 0
2016-11-03 16:32:28,413:INFO:PROGRESS: at 3.06% examples, 265721 words/s, in_qsize 7, out_qsize 0
2016-11-03 16:32:29,

Iteration: 9!!!!!!!!!!!!!!


2016-11-03 16:37:44,207:INFO:PROGRESS: at 0.26% examples, 224216 words/s, in_qsize 8, out_qsize 0
2016-11-03 16:37:45,229:INFO:PROGRESS: at 0.60% examples, 257376 words/s, in_qsize 7, out_qsize 0
2016-11-03 16:37:46,250:INFO:PROGRESS: at 0.91% examples, 261095 words/s, in_qsize 8, out_qsize 0
2016-11-03 16:37:47,261:INFO:PROGRESS: at 1.25% examples, 269175 words/s, in_qsize 8, out_qsize 0
2016-11-03 16:37:48,309:INFO:PROGRESS: at 1.59% examples, 273138 words/s, in_qsize 7, out_qsize 0
2016-11-03 16:37:49,347:INFO:PROGRESS: at 1.92% examples, 275431 words/s, in_qsize 8, out_qsize 0
2016-11-03 16:37:50,363:INFO:PROGRESS: at 2.22% examples, 274621 words/s, in_qsize 7, out_qsize 0
2016-11-03 16:37:51,371:INFO:PROGRESS: at 2.55% examples, 274474 words/s, in_qsize 8, out_qsize 0
2016-11-03 16:37:52,394:INFO:PROGRESS: at 2.86% examples, 273808 words/s, in_qsize 8, out_qsize 0
2016-11-03 16:37:53,398:INFO:PROGRESS: at 3.17% examples, 273760 words/s, in_qsize 8, out_qsize 0
2016-11-03 16:37:54,

In [3]:
#doc2Vec method
#when we train doc2vec, we can get word embeddings and also document similarities, 
#and even label representations!

#print len(model.docvecs)
#print len(model.index2word)
#model.docvecs[0]
#all_documents[75000].tags
#print model.docvecs

train_data_average=[model.docvecs[m] for m in range(25000)]
test_data_average=[model.docvecs[m] for m in range(75000,100000)]

In [7]:
from sklearn.linear_model import LogisticRegression
clf=LogisticRegression()
logistic=clf.fit(train_data_average,train["sentiment"])
result=logistic.predict_proba(test_data_average)[:,1]
output=pd.DataFrame(data={"id":test['id'],"sentiment":result})
output.to_csv("/Users/shichangtai/Desktop/Kaggle/Bag of Wprds Meet Bags of Popcorn/result_Doc2Vec.csv",index=False,quoting=3)
#score of 0.93169

In [1]:
#0.93169
from gensim.models.doc2vec import LabeledSentence, Doc2Vec, Word2Vec
model=Doc2Vec.load("Doc2VectforNLPTraining")
#print(model.most_similar(positive=['woman', 'king'], negative=['man'], topn=10))

/usr/local/lib/python2.7/site-packages/gensim/utils.py:1015: UserWarning: Pattern library is not installed, lemmatization won't be available.
  warnings.warn("Pattern library is not installed, lemmatization won't be available.")


In [4]:
model.most_similar("queen")

[(u'queens', 0.34981781244277954),
 (u'lady', 0.2893381714820862),
 (u'gees', 0.2574976980686188),
 (u'mistress', 0.25623852014541626),
 (u'heroine', 0.2559276819229126),
 (u'girl', 0.2538259029388428),
 (u'dude', 0.2527107000350952),
 (u'woman', 0.23100697994232178),
 (u'baby', 0.22838811576366425),
 (u'floodwater', 0.22670821845531464)]